In [7]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import torch

In [19]:
apple = Image.open('/Users/kymbradshaw/Desktop/test_images/orange.jpg')

In [20]:
processor_g = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
model_g = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

In [21]:
inputs = processor_g(apple, return_tensors="pt")

In [22]:
with torch.no_grad():
    logits = model_g(**inputs).logits


predicted_label = logits.argmax(-1).item()
print(model_g.config.id2label[predicted_label])

orange
